In [1]:
!pip3 install pandas h3 sqlalchemy boto3 oci

Defaulting to user installation because normal site-packages is not writeable


In [2]:
print("starting...")



# -*- coding: utf-8 -*-
"""part_1_mobility_script_generic(new_dataset)_new (2).ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1dGNHBaJAS1w2hsZw4c_Kx4bFdQ0Kzdg_
"""

print("starting...")

from pyspark.sql.window import Window 
import pyspark
from pyspark import SparkContext

from pyspark.sql import functions as F
import pyspark.sql.functions as func


from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import col, window
import pyspark.sql.functions as func
from pyspark.sql.functions import desc
import datetime, time
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.window import Window
import pyspark.sql.functions as sf

import datetime as dt
import numpy as np
import os

from multiprocessing.pool import Pool
import pickle
import pandas as pd

from h3 import h3


from math import radians, cos, sin, asin, sqrt
from pyspark.sql.functions import lit
import json

from datetime import datetime, date, timedelta
 
import oci
from pyspark.sql import SparkSession
import sys
import boto3
from io import BytesIO

from functools import reduce
from pyspark.sql import DataFrame

from pyspark.sql.functions import round, col
from dateutil import tz
import sqlalchemy as db

import pyspark

from pyspark.sql import SparkSession
ociconf = oci.config.from_file()

spark = SparkSession.builder \
        .config("spark.driver.memory", "4g") \
        .config("spark.driver.cores", "4") \
        .config("spark.executor.memory", "12g") \
        .config("spark.executor.cores", "4") \
        .config("spark.scheduler.mode", "FAIR") \
        .config("spark.cores.max", "16") \
        .getOrCreate()

# spark.conf.set('fs.oci.client.auth.tenantId', ociconf['tenancy'])
# spark.conf.set('fs.oci.client.auth.userId',  ociconf['user'])
# spark.conf.set('fs.oci.client.auth.fingerprint', ociconf['fingerprint'])
# spark.conf.set('fs.oci.client.auth.pemfilepath', "~/.oci/oci_api_key")
# spark.conf.set('fs.oci.client.auth.tenantId.region', "us-ashburn-1")
# spark.conf.set('fs.oci.client.hostname', "https://objectstorage.us-ashburn-1.oraclecloud.com")

spark.conf.set('fs.oci.client.auth.tenantId', "ocid1.tenancy.oc1..aaaaaaaamqxmfclvmrazpk4kt7ibkcfzfg2fvg4o2wyzemzu3n7tcpf6nvsa")
spark.conf.set('fs.oci.client.auth.userId', "ocid1.user.oc1..aaaaaaaa4gjumyz4kiowhzbyjzgajyo5bln565rntgboqod3t6lu4bpwl2ra")
spark.conf.set('fs.oci.client.auth.fingerprint', "ec:8f:0e:14:e0:31:cd:e4:d6:03:04:aa:aa:94:f9:a9")
spark.conf.set('fs.oci.client.auth.pemfilepath', "~/.oci/oci_api_key")
spark.conf.set('fs.oci.client.auth.tenantId.region', "us-ashburn-1")
spark.conf.set('fs.oci.client.hostname', "https://objectstorage.us-ashburn-1.oraclecloud.com")

# spark.driver.cores
# spark.driver.memory
# spark.driver.memoryOverhead
# spark.executor.memory = 1g
# spark.executor.memoryOverhead
# spark.driver.resource.{resourceName}.amount
# spark.executor.resource.{resourceName}.amount
# spark.executor.cores
spark.sparkContext._conf.getAll()  
#         .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
# .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

starting...
starting...


/opt/spark/python/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


[('spark.driver.cores', '4'),
 ('spark.driver.memory', '4g'),
 ('spark.driver.port', '44808'),
 ('spark.scheduler.mode', 'FAIR'),
 ('spark.cores.max', '16'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'app-20220411101946-0012'),
 ('spark.sql.warehouse.dir',
  'file:/home/centos/work/mobility_eval/spark-warehouse'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.executor.memory', '12g'),
 ('spark.executor.cores', '4'),
 ('spark.driver.host',
  'oke-cf2oghpidfa-nt5qvktoxqa-shjnqmwa77q-1.subc6544f31f.useastcluster.oraclevcn.com'),
 ('spark.app.startTime', '1649672384886'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.master', 'spark://10.0.10.62:7077'),
 ('spark.ui.showConsoleProgress', 'true')]

In [13]:
save_string = f"oci://mobility-data@bmmp5bv7olp2/usa/foursquare/delivery_dt=2022-02-02/SVF/visits/dt=2022-01-15/country=US/FSQ-SVF-US-2022-01-15.tsv.gz"
df1 = spark.read.option("delimiter", "\t").format("csv").load(save_string, header = 'true')

save_string = f"oci://mobility-data@bmmp5bv7olp2/usa/foursquare/delivery_dt=2022-02-02/SVF/visits/dt=2022-01-16/country=US/FSQ-SVF-US-2022-01-16.tsv.gz"
df2 = spark.read.option("delimiter", "\t").format("csv").load(save_string, header = 'true')

# save_string = f"oci://mobility-data@bmmp5bv7olp2/usa/startapp/location/20220328"
# df1 = spark.read.format("csv").load(save_string, header = 'false')

# save_string2 = f"oci://mobility-data@bmmp5bv7olp2/usa/startapp/location/20220329"
# df2 = spark.read.format("csv").load(save_string2, header = 'false')

df = df1.union(df2)
# cols = ['mobile_ad_id',  'longitude', "latitude", "accuracy", "timestamp","source", "IP"]
# df = df.toDF(*cols)
df.columns

['maid',
 'protected_timestamp',
 'chainid',
 'chainname',
 'postalcode',
 'venueid',
 'source_indicator',
 'dwell_minutes',
 'confidence_decile',
 'home_postalcode',
 'venue_latitude',
 'venue_longitude',
 'year_month']

In [14]:
df.columns

['maid',
 'protected_timestamp',
 'chainid',
 'chainname',
 'postalcode',
 'venueid',
 'source_indicator',
 'dwell_minutes',
 'confidence_decile',
 'home_postalcode',
 'venue_latitude',
 'venue_longitude',
 'year_month']

In [15]:
df = df.withColumnRenamed("maid", "mobile_ad_id")
df = df.withColumnRenamed("protected_timestamp", "timestamp")
# df = df.withColumnRenamed("h3index12", "h12")
# df = df.withColumnRenamed("h3index9", "h3")
df = df.withColumnRenamed("venue_longitude", "longitude")
df = df.withColumnRenamed("venue_latitude", "latitude")


In [8]:

master_data = spark.read.option("header", "true").csv("oci://ia-datapipeline@bmmp5bv7olp2/location_master_data/US")
master_data.show(3)

+-------+-------+-----+---------------+
|country|  state| city|           idx9|
+-------+-------+-----+---------------+
|     US|Montana|30035|8912c86caabffff|
|     US|Montana|30101|8912c901b1bffff|
|     US|Montana|30101|8912c904137ffff|
+-------+-------+-----+---------------+
only showing top 3 rows



In [9]:
master_data =master_data.withColumnRenamed("idx9", "h3index")

In [16]:
h3_func9 = lambda lat, lng: h3.geo_to_h3(float(lat), float(lng), resolution =9)
h3_full_function9 = F.udf(h3_func9)
df = df.withColumn("h3index", h3_full_function9(*[F.col(x) for x in ["latitude", "longitude"]]))

In [21]:
dfi_places = df.join(master_data, on ="h3index", how="left")
dfi_places.select([F.count(F.when(F.col(c).isNotNull() , c)).alias(c) for c in dfi_places.columns]
   ).show()


+--------+------------+---------+--------+---------+----------+--------+----------------+-------------+-----------------+---------------+--------+---------+----------+--------+--------+--------+
| h3index|mobile_ad_id|timestamp| chainid|chainname|postalcode| venueid|source_indicator|dwell_minutes|confidence_decile|home_postalcode|latitude|longitude|year_month| country|   state|    city|
+--------+------------+---------+--------+---------+----------+--------+----------------+-------------+-----------------+---------------+--------+---------+----------+--------+--------+--------+
|17399678|    17399678| 17399678|17399678| 17399678|  17399678|17399678|        17399678|     17399678|         17399678|       13931321|17399678| 17399678|  17399678|17381052|17381052|17381052|
+--------+------------+---------+--------+---------+----------+--------+----------------+-------------+-----------------+---------------+--------+---------+----------+--------+--------+--------+



In [22]:
country = "US"

In [23]:
df_coverage = dfi_places.groupby(["state","city"]).agg(
    F.countDistinct("mobile_ad_id").alias("num_devices"),
     F.count(F.lit(1)).alias("num_readings"),
     )
df_coverage.coalesce(1).write.format("csv").mode('overwrite').option("header", "true").save("oci://processed-data@bmmp5bv7olp2/foursquare/"+country+"/13_1.csv")


In [24]:
df.select("mobile_ad_id").distinct().count()

8532734

In [ ]:
save_string = f"oci://mobility-data@bmmp5bv7olp2/usa/foursquare/delivery_dt=2022-02-02/SVF/visits/dt=2022-01-15/country=US/FSQ-SVF-US-2022-01-15.tsv.gz"
df1 = spark.read.option("delimiter", "\t").format("csv").load(save_string, header = 'true')

save_string = f"oci://mobility-data@bmmp5bv7olp2/usa/foursquare/delivery_dt=2022-02-02/SVF/visits/dt=2022-01-16/country=US/FSQ-SVF-US-2022-01-16.tsv.gz"
df2 = spark.read.option("delimiter", "\t").format("csv").load(save_string, header = 'true')

# save_string = f"oci://mobility-data@bmmp5bv7olp2/usa/startapp/location/20220328"
# df1 = spark.read.format("csv").load(save_string, header = 'false')

# save_string2 = f"oci://mobility-data@bmmp5bv7olp2/usa/startapp/location/20220329"
# df2 = spark.read.format("csv").load(save_string2, header = 'false')

df = df1.union(df2)
# cols = ['mobile_ad_id',  'longitude', "latitude", "accuracy", "timestamp","source", "IP"]
# df = df.toDF(*cols)
df.columns